### Medical procedures by physician, Medicare data (2012)
Janos Perge, 07/21/2016

Purpose:   
1) Access Medicare data on the number of performed medical procedures per physician (as a proxi for quality of care).   
2) Convert HCPCS (or CPT) procedure codes to CCS codes for further analysis  
3) Save data in three spreadsheets: #1-Provider information, #2-CPT-to-CCS conversion table and #3-Number of performed procedures per provider, broken down to different CCS categories (244 different procedure types)
    
Before running this code, first visit the url below, accept CMS disclaimer, download and unzip file (2GB) and place it within the same directory as this script. Running this code first time took XXXX min on my Win10 machine with core i7 and 8GB RAM.   

Data is a public use file downloadable from CMS:
https://www.cms.gov/apps/ama/license.asp?file=http://download.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Medicare_Provider_Util_Payment_PUF_CY2012_update.zip


#### Obtain packages

In [2]:
packageList = c("data.table","stringr",'plyr')

is_installed <- function(mypkg) is.element(mypkg, installed.packages()[,1])

load_or_install<-function(package_names)
{
  for(package_name in package_names)
  {
    if(!is_installed(package_name))
    {
      install.packages(package_name,repos="http://lib.stat.cmu.edu/R/CRAN")
    }
    options(java.parameters = "-Xmx8g")
    library(package_name,character.only=TRUE,quietly=TRUE,verbose=FALSE)
  }
}

load_or_install(packageList)

#### Access data

In [3]:
cms_filename = "Medicare_Provider_Util_Payment_PUF_CY2012.txt" #available data for years 2012,2013,2014
my_data_file = "procedures2012.RData"

start = Sys.time()
#open data from tabular file saved on HD or from Rdatafile:
if(file.exists(my_data_file) && !exists("physician_data")){
  load(my_data_file)
} else if(!file.exists(my_data_file)) {
  physician_data = read.delim(physician_medicare, stringsAsFactors=FALSE)
  physician_data = physician_data[2:nrow(physician_data),]
  colnames(physician_data) = tolower(colnames(physician_data))

  save(physician_data, file=my_data_file)
}
#physician_data = data.table(physician_data)
    
Sys.time()-start

Time difference of 1.062612 mins

In [4]:
head(physician_data)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,stdev_medicare_allowed_amt,average_submitted_chrg_amt,stdev_submitted_chrg_amt,average_medicare_payment_amt,stdev_medicare_payment_amt
2,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,115,112,115,135.25,0,199,0,108.11565217,0.9005883395
3,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,93,88,93,198.59,0,291,9.5916630466,158.87,0
4,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,111,83,111,38.75,0,58,0,30.720720721,2.9291057922
5,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,544,295,544,70.95,0,105,0,56.655661765,2.4313271619
6,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,75,55,75,101.74,0,150,0,81.39,0
7,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,95,95,95,71.06,0,104,0,55.768842105,7.4155003896


In [5]:
colnames(physician_data)

[1] "npi"                              "nppes_provider_last_org_name"    
 [3] "nppes_provider_first_name"        "nppes_provider_mi"               
 [5] "nppes_credentials"                "nppes_provider_gender"           
 [7] "nppes_entity_code"                "nppes_provider_street1"          
 [9] "nppes_provider_street2"           "nppes_provider_city"             
[11] "nppes_provider_zip"               "nppes_provider_state"            
[13] "nppes_provider_country"           "provider_type"                   
[15] "medicare_participation_indicator" "place_of_service"                
[17] "hcpcs_code"                       "hcpcs_description"               
[19] "hcpcs_drug_indicator"             "line_srvc_cnt"                   
[21] "bene_unique_cnt"                  "bene_day_srvc_cnt"               
[23] "average_medicare_allowed_amt"     "stdev_medicare_allowed_amt"      
[25] "average_submitted_chrg_amt"       "stdev_submitted_chrg_amt"        
[27] "average_medicare_payment_amt"     "stdev_medicare_payment_amt"

In [56]:
#physician data is large and clogs memory. Therefore I take what I need and clear the rest from the workspace:
descriptor_vars = c("npi", "nppes_provider_last_org_name", "nppes_provider_first_name", "nppes_provider_mi", 
                    "nppes_credentials", "nppes_provider_gender", "nppes_entity_code", "nppes_provider_street1", 
                    "nppes_provider_street2", "nppes_provider_city", "nppes_provider_zip", "nppes_provider_state", 
                    "nppes_provider_country", "provider_type", "medicare_participation_indicator", "place_of_service")

physician_info   = physician_data[, descriptor_vars]
doctor_procedure = physician_data[, c('npi', 'hcpcs_code','line_srvc_cnt' )]
conversion_table = physician_data[, c('hcpcs_code', 'hcpcs_description')]
#rm(physician_data)

#### Spreadsheet #1: Doctor's parameters such as NPI, Name, state, gender, etc...

In [78]:
physician_info = physician_info[!duplicated(physician_info$npi),]  # no time saved by converting to a data.table 

#### Spreadsheet #2: HCPCS/CPT code to CCS conversion

In [57]:
conversion_table = conversion_table[!duplicated(conversion_table$hcpcs_code),]
conversion_table = data.table(conversion_table)
setkey(conversion_table, hcpcs_code)
head(conversion_table)

,hcpcs_code,hcpcs_description
1,00100,Anesthesia for procedure on salivary gland with biopsy
2,00102,Anesthesia for procedure to repair lip defect present at birth
3,00103,Anesthesia for procedure on eyelid
4,00104,Anesthesia for electric shock treatment
5,00120,Anesthesia for biopsy of external middle and inner ear
6,00126,Anesthesia for incision of ear drum


In [58]:
ccs_file = '2016_ccs_services_procedures.csv'
ccs_table = data.table(read.csv(ccs_file))
ccs_table$Code.Range <- as.character(ccs_table$Code.Range)
setkey(ccs_table,Code.Range,CCS,CCS.Label)
head(ccs_table)

,Code.Range,CCS,CCS.Label
1,'0001T-0002T',52,"Aortic resection, replacement or anastomosis"
2,'0003T-0003T',130,"Other diagnostic procedures, female organs"
3,'0005T-0006T',59,Other OR procedures on vessels of head and neck
4,'0007T-0007T',211,Therapeutic radiology
5,'0008T-0008T',93,Other non-OR upper GI therapeutic procedures
6,'0009T-0009T',125,Other excision of cervix and uterus


In [59]:
# combine the above two tables

In [60]:
#create an incremental sequence of CSS codes from Code.Range:
get_code_range <- function(inp,ccscode,ccsdesc){
    
    code_range <- vector(mode="numeric", length=0) #empty 
    
    aaa = unlist(strsplit(inp, "-", fixed = TRUE))    
    aaa = sub("\'", "", aaa)
    
    if (!grepl("[a-zA-Z]", aaa[1])){  #if code does not contain letters
        aaa = as.numeric(aaa)
        code_range = seq.int(aaa[1],aaa[2])
        code_range = sprintf("%05d", code_range) # fixed width of five characters with leading zeros
        code_range = as.character(code_range)
        
    } else #if hcpcs code is alphanumeric, with the numeric part as an incremental sequence
    {
        bbb = substring(aaa[1], seq(1,nchar(aaa[1])), seq(1,nchar(aaa[1]),1)) #break up string to individual characters
        letterPos = grep("[a-zA-Z]", bbb, value = FALSE)
        letterChar = grep("[a-zA-Z]", bbb, value = TRUE)

        numericPart1 = grep("[0-9]", bbb, value = TRUE)
        numericPart1 = as.numeric(paste(numericPart1, collapse=""))

        bbb = substring(aaa[2], seq(1,nchar(aaa[2])), seq(1,nchar(aaa[2]),1)) 
        numericPart2 = grep("[0-9]", bbb, value = TRUE)
        numericPart2 = as.numeric(paste(numericPart2, collapse=""))

        cr  = seq.int(numericPart1,numericPart2)
        
        if (letterPos==1){
            code_range = sprintf("%s%04d", letterChar, cr) # fixed width of four characters with leading zeros
        } else
        {
            code_range = sprintf("%04d%s", cr, letterChar)
        }            
    }
    out = data.frame(hcpcs_code=code_range, ccs_code = ccscode, css_desc = ccsdesc)
}

In [62]:
# Expand CCS table, i.e. list every HCPCS code specified within the Code.Range
expanded_ccs = with(ccs_table, Map(get_code_range, Code.Range, CCS, CCS.Label))
expanded_ccs = rbind.fill(expanded_ccs)
expanded_ccs = data.table(expanded_ccs) 
setkey(expanded_ccs,hcpcs_code)
conversion_table = merge(conversion_table,expanded_ccs, by='hcpcs_code')

In [61]:
# This code would be much faster, but factor conversion is still unresolved:
# Expand CCS table, i.e. print every HCPCS code specified within the Code.Range
# expanded_ccs = data.frame(ccs_table[, get_code_range(Code.Range, CCS, CCS.Label), by = 1:nrow(ccs_table)])
# expanded_css$hcpcs_code <- as.character(expanded_css$hcpcs_code)
# ccs_table$Code.Range <- as.character(ccs_table$Code.Range)
# expanded_ccs = data.table(expanded_ccs)
# expanded_css = expanded_ccs[,nrow:=NULL]
# setkey(expanded_css,hcpcs_code)
# head(expanded_css)

In [63]:
head(conversion_table)

,hcpcs_code,hcpcs_description,ccs_code,css_desc
1,00100,Anesthesia for procedure on salivary gland with biopsy,232,Anesthesia
2,00102,Anesthesia for procedure to repair lip defect present at birth,232,Anesthesia
3,00103,Anesthesia for procedure on eyelid,232,Anesthesia
4,00104,Anesthesia for electric shock treatment,232,Anesthesia
5,00120,Anesthesia for biopsy of external middle and inner ear,232,Anesthesia
6,00126,Anesthesia for incision of ear drum,232,Anesthesia


#### Spreadsheet #3: Provider vs Procedure count.  
Rows: provider (npi as an ID), columns: CCS code.  
cells in the final matrix (npi_vs_css) correspond to total procedure counts for a given provider and given CCS category

In [65]:
# Merge CCS codes into physician data frame:
toAppend = conversion_table[, .(hcpcs_code ,ccs_code)]
setkey(toAppend,hcpcs_code)

In [66]:
head(toAppend)

,hcpcs_code,ccs_code
1,00100,232
2,00102,232
3,00103,232
4,00104,232
5,00120,232
6,00126,232


In [67]:
doctor_procedure = data.table(doctor_procedure)
setkey(doctor_procedure, "hcpcs_code")
doctor_procedure = merge(doctor_procedure, toAppend, by='hcpcs_code')

In [68]:
head(doctor_procedure)

,hcpcs_code,npi,line_srvc_cnt,ccs_code
1,00100,1396897104,14,232
2,00100,1508884156,12,232
3,00100,1528032067,14,232
4,00100,1780646562,13,232
5,00102,1700846235,17,232
6,00103,1003019191,14,232


In [69]:
start = Sys.time()

setkey(doctor_procedure, "npi","ccs_code")
npi_vs_css = doctor_procedure[, .(proc_count = sum(line_srvc_cnt)), by = .(npi,ccs_code)] #pool similar procedures (with identical ccs codes)
setkey(npi_vs_css, "npi","ccs_code")
npi_vs_css= dcast(npi_vs_css, npi ~ ccs_code) #convert data.table from long to wide format (i.e. npi vs. ccs table)

Sys.time()-start

Using 'proc_count' as value column. Use 'value.var' to override


Time difference of 15.62347 secs

In [70]:
head(npi_vs_css)

,npi,1,2,3,4,5,6,7,8,9,...,233,234,235,237,239,240,241,242,243,244
1,1003000126,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1003000134,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,7673,NA,NA,NA,NA,NA,NA,NA,NA
3,1003000142,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1003000381,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,1003000407,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1003000423,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,11,NA,NA,NA,NA,NA,NA,NA,NA


#### Save results

In [75]:
#csv files:
start = Sys.time()
# write.csv(physician_info, file = "physician_info.csv", row.names=FALSE, na="")
# write.csv(conversion_table, file = "CPT_to_CCS_conversion.csv", row.names=FALSE, na="")
# write.csv(npi_vs_css, file = "provider_vs_procedures2012.csv", row.names=FALSE, na="")

#Note that write.csv saves numeric hcpcs codes as an arabic number e.g. '102', omiting zero characters in the beginning.
#This would cause mismatches later when mapping fixed-length character arrays such as '00102'. 
#However, R's native data format saves the codes with leading zeros.

#R.data file:
save(physician_info, conversion_table, npi_vs_css, file='provider_vs_procedures2012.RData')

Sys.time()-start

Time difference of 1.470248 mins